In [20]:
import datetime
import glob
import os

import numpy as np
import pandas as pd

import rasterio
import cartopy
import shapely
import geopandas as gpd

import matplotlib.pyplot as plt

# Plot some risk and hazard maps

# 1. Constants for plotting

In [21]:
#raster resolution in meters
pixel_size=10000

#bounds of output grid in lat/lon
output_min_lon = -168
output_max_lon = -141
output_min_lat = 69
output_max_lat = 73

# Lat/Lon CRS
in_crs = {'init': 'epsg:4326'}

# Equal area projection
out_crs={'init': 'epsg:3338'}

In [22]:
# convert the bounding box to our output projection
bounding_box = shapely.geometry.box(output_min_lon, output_min_lat, output_max_lon, output_max_lat)
bbDF = gpd.GeoDataFrame(crs=in_crs,geometry=[bounding_box])
bbDF = bbDF.to_crs(out_crs)

In [23]:
x_min, y_min, x_max, y_max = bbDF.iloc[0].geometry.bounds
x_res = int((x_max - x_min) / pixel_size)
y_res = int((y_max - y_min) / pixel_size)
print('Output dimensions: ' + str(x_res) + ' x ' + str(y_res))

Output dimensions: 108 x 43


In [24]:
# map features
crs = cartopy.crs.epsg(3338)

land_10m = cartopy.feature.NaturalEarthFeature('physical', 'land', '10m',
                                        edgecolor='gray',
                                        facecolor=cartopy.feature.COLORS['land'])

# 2. Calculate risk

Following Sepp-Neves, risk $R$ is calculated as

$R_s = H_s * I_s$

where $H$ is the oil spill hazard and $I$ is the vulnerability.  Here we treat each species as a seperate vulernability.  In Sepp-Neves $H_s$ was integrated across each region $s$ then multiplied by the scalar $I_s$.  Here we have more ganular data, so instead we integrate $R_s$ across each region $s$.

In [25]:
out_dir = '/data/assets/arctic-osra/2017_spills/updated_results'

In [26]:
ship_types = ['anti-pollution',
              'cargo',
              'icebreaker',
              'landing',
              'offshoresupply',
              'passenger',
              'recreational'
              'survey',
              'tanker',
              'tugboat',
             ]

communities = ['kaktovik',
               'nuiqsut',
               'utqiagvik'
              ]

month_map = {i: datetime.datetime(2017, i, 1).strftime('%B') for i in range(1, 13)}


species = ['arcticcisco',
           'beardedseal',
           'bowhead',
           'broadwhitefish',
           'burbot',
           'caribou',
           'dollyardenchar',
           'eider',
           'geese',
           'moose',
           'ringedseal',
           'walrus',
           'wolfandwolverine']

In [27]:
risk_score = pd.DataFrame({'community': [],
                           'ship_type': [],
                           'month': [],
                           'species': [],
                           'risk_score': []})

# don't need the next cell, but keeping in case I want things later

calc = True
if calc:
    for ship_type in ship_types:
        for community in communities:
            for month in range(7, 12):
                try:
                    fname1 = os.path.join(out_dir, 'rs-{}-{}-{}.npy'.format(ship_type,
                                                                            community,
                                                                               month_map[month].lower()))
                    fspecies = glob.glob(fname1)
                    species = set([os.path.basename(f).split(community+'-')[1].split('-')[0] for f in fspecies])
                    for s in species:
                        fname = 'monthly_is_{}-{}-{}-{}.tiff'.format(ship_type,
                                                                     community,
                                                                     s,
                                                                     month_map[month].lower()
                                                                    )
                        fpath = os.path.join(out_dir, fname)
                        rs = 
                            data = ds.read()
                            rs = data.sum()
                            risk_score = risk_score.append({'community': community,
                                                            'ship_type': ship_type,
                                                            'month': month,
                                                            'species': s,
                                                            'risk_score': data.sum()}, ignore_index=True)
                        
    risk_score.to_csv('../../outputs/OSRA/risk_score.csv')
else:
    risk_score = pd.read_csv('../../outputs/OSRA/risk_score.csv')
    risk_score = risk_score.loc[:, ~risk_score.columns.str.contains('^Unnamed')]
    
risk_score.head()
community_rs = risk_score.groupby('community')
annual_max = community_rs.sum().max().risk_score

### 2.0.1 Calculate maximum monthly risk by community

In [28]:
def calc_community_total_annual_risk(out_dir, community):
    """Calculate annual risk for community"""
    files = glob.glob(out_dir + '/rs-*-{}-*.npy'.format(community))
    files.sort()

    total_risk = 0
    for f in files:
        tmp = np.load(f).sum()
        total_risk += tmp

    return total_risk

def calc_max_annual_risk(out_dir, commmunities):
    """Returns max risk for all communities"""
    max_risk = 0
    for community in communities:
        comm_risk = calc_community_total_annual_risk(out_dir, community)
        if comm_risk > max_risk:
            max_risk = comm_risk

    return max_risk

max_annual_risk = calc_max_annual_risk(out_dir, communities)

In [29]:
max_annual_risk
annual_max = max_annual_risk

## 2.1 Annual risk by community

In [30]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs-2017/community'

In [31]:
%%time
for community in communities:
    print(community)
    fname = os.path.join(out_dir + '/rs-*-{}-*.tif'.format(community))
    print('- fname: {}'.format(fname))
    files = glob.glob(fname)
    files.sort()
    
    # set up data size
    with rasterio.open(files[0]) as ds:
        profile = ds.profile
        data = ds.read()
    data = np.zeros_like(data)
    
    # change dtype
    profile.update(dtype=rasterio.float32, count=1)
    
    for f in files:
        with rasterio.open(f) as ds:
            tmp = ds.read()
            tmp = np.nan_to_num(tmp)
            data += tmp
            #print('- {} {} {}'.format(os.path.basename(f), np.nansum(tmp), np.nansum(data)))
    print(np.nansum(data))
    data = data/annual_max
    print(np.nansum(data))

    # save tiff
    out_path = os.path.join(tif_out_dir, '{}-allresources-allmonths.tif'.format(community.lower()))
    with rasterio.open(out_path, 'w', **profile) as out:
        out.write(data[0, :, :].astype(rasterio.float32), 1)

kaktovik
- fname: /data/assets/arctic-osra/2017_spills/updated_results/rs-*-kaktovik-*.tif
1.3993413
0.065797776
nuiqsut
- fname: /data/assets/arctic-osra/2017_spills/updated_results/rs-*-nuiqsut-*.tif
9.900242
0.4655146
utqiagvik
- fname: /data/assets/arctic-osra/2017_spills/updated_results/rs-*-utqiagvik-*.tif
21.267303
0.9999999
CPU times: user 6.35 s, sys: 2.71 s, total: 9.06 s
Wall time: 4min 3s


In [32]:
# set up data size
with rasterio.open(files[0]) as ds:
    example_profile = ds.profile
    data = ds.read()
example_data = np.zeros_like(data)
    
# change dtype
example_profile.update(dtype=rasterio.float32, count=1)

## 2.2 Monthly risk by community

### 2.2.1 Make tifs

In [33]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs-2017/community-month'

In [34]:
for community in communities:
    print(community)
    for month in range(1, 13):
        month_name = month_map[month]
        fname = os.path.join(out_dir + '/rs-*-{0}-{1:02d}-*.tif'.format(community, month))
        files = glob.glob(fname)
        files.sort()

        # set up data size
        data = np.zeros_like(example_data)

        for f in files:
            with rasterio.open(f) as ds:
                tmp = ds.read()
                tmp = np.nan_to_num(tmp)
                data += tmp
        data = data/annual_max
        print('- {} {}'.format(month, np.nansum(data)))

        # save tiff
        out_path = os.path.join(tif_out_dir, '{}-allresources-{}.tif'.format(community.lower(), month_name.lower()))
        with rasterio.open(out_path, 'w', **example_profile) as out:
            out.write(data[0, :, :].astype(rasterio.float32), 1)

kaktovik
- 1 0.0
- 2 0.0
- 3 0.0
- 4 0.0
- 5 0.0
- 6 0.0
- 7 0.0016994294710457325
- 8 0.005057556554675102
- 9 0.055354196578264236
- 10 0.003686592448502779
- 11 0.0
- 12 0.0
nuiqsut
- 1 0.0
- 2 0.0
- 3 0.0
- 4 0.0
- 5 7.096247281879187e-05
- 6 0.0011569422204047441
- 7 0.034066092222929
- 8 0.058491408824920654
- 9 0.33525967597961426
- 10 0.03646952658891678
- 11 0.0
- 12 0.0
utqiagvik
- 1 0.0
- 2 0.0
- 3 0.0
- 4 0.0
- 5 0.0
- 6 0.007250379770994186
- 7 0.14513778686523438
- 8 0.225322425365448
- 9 0.22009247541427612
- 10 0.37645256519317627
- 11 0.025744352489709854
- 12 0.0


## 2.3 Annual risk by community, species

Normalized by maximum annual risk of all species

In [35]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs-2017/community-resource'

In [36]:
for community in communities:
    print(community)
    for s in species:
        fname = os.path.join(out_dir + '/rs-*-{}-*-{}.tif'.format(community, s))
        files = glob.glob(fname)
        files.sort()
        print('- ' + s + ' ' + str(len(files)))

        # set up data size
        data = np.zeros_like(example_data)

        for f in files:
            with rasterio.open(f) as ds:
                tmp = ds.read()
                tmp = np.nan_to_num(tmp)
                data += tmp
        data = data/annual_max
        print('-- {}'.format(np.nansum(data)))

        # save tiff
        out_path = os.path.join(tif_out_dir, '{}-{}-allmonths.tif'.format(community.lower(), s))
        with rasterio.open(out_path, 'w', **example_profile) as out:
            out.write(data[0, :, :].astype(rasterio.float32), 1)

kaktovik
- arcticcisco 48
-- 0.00012231364962644875
- beardedseal 48
-- 0.007086501456797123
- bowhead 48
-- 0.04580312967300415
- broadwhitefish 48
-- 0.00012231364962644875
- burbot 48
-- 0.00012231364962644875
- caribou 48
-- 0.002692857990041375
- dollyardenchar 0
-- 0.0
- eider 48
-- 4.727693158201873e-05
- geese 48
-- 0.00019783712923526764
- moose 48
-- 0.0
- ringedseal 48
-- 0.006991541478782892
- walrus 48
-- 0.00222886074334383
- wolfandwolverine 48
-- 1.5888183042989112e-05
nuiqsut
- arcticcisco 48
-- 0.0
- beardedseal 48
-- 0.09841542690992355
- bowhead 48
-- 0.2556510269641876
- broadwhitefish 48
-- 0.0
- burbot 48
-- 0.0
- caribou 48
-- 0.0029155598022043705
- dollyardenchar 0
-- 0.0
- eider 48
-- 0.01601129025220871
- geese 48
-- 0.0
- moose 48
-- 0.0
- ringedseal 48
-- 0.09222956746816635
- walrus 48
-- 0.0002917099336627871
- wolfandwolverine 48
-- 0.0
utqiagvik
- arcticcisco 48
-- 0.0004881578788626939
- beardedseal 48
-- 0.192459836602211
- bowhead 48
-- 0.4608794450

## 2.4 Risk score by species, month

In [37]:
tif_out_dir = '../../outputs/OSRA/spatial-risk-tifs-2017/community-resource-month'

In [38]:
for community in communities:
    print(community)
    for s in species:
        print('- ' + s)
        for month in range(1, 13):

            month_name = month_map[month]
            fname = os.path.join(out_dir + '/rs-*-{0}-{1:02d}-{2}.tif'.format(community, month, s))
            files = glob.glob(fname)
            files.sort()

            # set up data size
            data = np.zeros_like(example_data)

            for f in files:
                with rasterio.open(f) as ds:
                    tmp = ds.read()
                    tmp = np.nan_to_num(tmp)
                    data += tmp
            data = data/annual_max
            print ('-- ' + str(month) + ' ' + str(len(files)) + ' ' + str(np.nansum(data)))
            
            # save tiff
            out_path = os.path.join(tif_out_dir, '{}-{}-{}.tif'.format(community.lower(), s, month_name.lower()))
            with rasterio.open(out_path, 'w', **example_profile) as out:
                out.write(data[0, :, :].astype(rasterio.float32), 1)

kaktovik
- arcticcisco
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.0
-- 8 10 0.0
-- 9 10 0.00012231365
-- 10 10 0.0
-- 11 3 0.0
-- 12 0 0.0
- beardedseal
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.00082227925
-- 8 10 0.0021013964
-- 9 10 0.004162825
-- 10 10 0.0
-- 11 3 0.0
-- 12 0 0.0
- bowhead
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.0
-- 8 10 0.00059656665
-- 9 10 0.04247418
-- 10 10 0.0027323803
-- 11 3 0.0
-- 12 0 0.0
- broadwhitefish
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.0
-- 8 10 0.0
-- 9 10 0.00012231365
-- 10 10 0.0
-- 11 3 0.0
-- 12 0 0.0
- burbot
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.0
-- 8 10 0.0
-- 9 10 0.00012231365
-- 10 10 0.0
-- 11 3 0.0
-- 12 0 0.0
- caribou
-- 1 0 0.0
-- 2 0 0.0
-- 3 0 0.0
-- 4 0 0.0
-- 5 1 0.0
-- 6 4 0.0
-- 7 10 0.0
-- 8 10 0.00012811125
-- 9 10 0.0022787594
-- 10 10 0.0002859